# ModelBuilder Inference Component

This notebook was tested with the `conda_python3` kernel on an Amazon SageMaker notebook instance of type `g5.2xl` with a 100GB EBS volume attached. If you are not using local mode, feel free to make use of a smaller CPU instance type / EBS volume. 

In [ ]:
!pip install boto3 sagemaker -U --quiet

# SageMaker Model Builder experience

In the new experience, we have introduced a few new constructs. Here we will focus on the following: 

1. ModelBuilder
2. SchemaBuilder
3. InferenceSpec

In the following section, we will define these constructs and provide examples to elaborate on each one.

4.1 ModelBuilder:

ModelBuilder is a Python class that takes a framework model (such as XGBoost or PyTorch) or an Inference Spec (more details below) and converts them into a SageMaker deployable model. ModelBuilder provides a `build` function that generates the artifacts for deployment. The model artifact generated is specific to the model server, which is also customizable as one of the inputs.

```python
Class definition:

class ModelBuilder(
    model_path: str | None = '/tmp/sagemaker/model-builder/' + uuid.uuid1().hex,
    role_arn: str | None = None,
    sagemaker_session: Session | None = None,
    name: str | None = 'model-name-' + uuid.uuid1().hex,
    mode: Mode | None = Mode.SAGEMAKER_ENDPOINT,
    shared_libs: List[str] = lambda : [],
    dependencies: Dict[str, Any] | None = lambda : { "auto": False },
    env_vars: Dict[str, str] | None = lambda : {},
    log_level: int | None = logging.DEBUG,
    content_type: str | None = None,
    accept_type: str | None = None,
    s3_model_data_url: str | None = None,
    instance_type: str | None = "ml.c5.xlarge",
    schema_builder: str | None = None,
    model: Any | None = None,
    inference_spec: InferenceSpec = None,
    image_uri: str | None = None,
    model_server: str | None = None
)
```
Example:

The above class file provide all the options for customization. However to deploy the framework model, the model builder just expects model, input, output and the role. 

```python
model_builder = ModelBuilder(
    model=model,  # Pass in the actual model object. It's "predict" method will be invoked in the endpoint.
    schema_builder=SchemaBuilder(input, output), # Pass in a "SchemaBuilder" which will use the sample test input and output objects to infer the serialization needed.
    role_arn=role, # Pass in the role arn or update intelligent defaults.
    )
```

4.2 SchemaBuilder:

The SchemaBuilder enables you to define the input and output for your endpoint. It allows the SchemaBuilder to generate the corresponding marshalling functions for serializing and deserializing the input and output. For further details, please consult the notebook or refer to the video.

Class definition:
```python
class SchemaBuilder(
    sample_input: Any,
    sample_output: Any,
    input_translator: CustomPayloadTranslator = None,
    output_translator: CustomPayloadTranslator = None
)
```
Example:

The CustomPayloadTranslator class provides all the options for customization. However, for [common inference data format](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html), you can just provide the sample input/output for the SchemaBuilder.
```python
input = "How is the demo going?"
output = "Comment la démo va-t-elle?"
schema = SchemaBuilder(input, output)
```

4.3 InferenceSpec

In the case you want to specify custom function to load and invoke the model instead of the framework model function, then you can pass the inference spec with your implementation in `load` and `invoke` function. 

class definition:
```python
class InferenceSpec(abc.ABC):
    @abc.abstractmethod
    def load(self, model_dir: str):
        pass

    @abc.abstractmethod
    def invoke(self, input_object: object, model: object):
        pass
```
Example:
```python
class MyInferenceSpec(InferenceSpec):
    def load(self, model_dir: str):
        return pipeline("translation_en_to_fr", model="t5-small")
        
    def invoke(self, input, model):
        return model(input)
   
inf_spec = MyInferenceSpec()

```

In this example, we are using ModelBuilder to deploy an Llama 2 model directly. You can use `Mode` to switch between local testing and deploying to a SageMaker Endpoint. 

### SageMaker ModelBuilder: Local deployment

Now we will use SageMaker ModelBuilder class to prepare the model for local and remote deployment.

In [ ]:
from sagemaker import get_execution_role, Session, image_uris
import boto3
import os
sagemaker_session = Session()
region = boto3.Session().region_name

# get execution role
# please use execution role if you are using notebook instance or update the role arn if you are using a different role
execution_role = get_execution_role() if get_execution_role() is not None else "your-role-arn"

In [ ]:
path = os.path.join(os.getcwd(), 'llama2-7b') 
os.makedirs(path, exist_ok=True)
print(path)

[Llama 2 is a gated model](https://huggingface.co/meta-llama/Llama-2-7b-hf) and requires access to be approved. Once approved you will need to pass your [Hugging Face access token](https://huggingface.co/docs/hub/security-tokens) as seen in the below cell. 

In [ ]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from sagemaker.serve import Mode
import json

prompt = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the"
response = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the east coast."

sample_input = {
    "inputs": prompt,
    "parameters": {}
}

sample_output = [
    {
        "generated_text": response
    }
]

model_builder_1 = ModelBuilder(
    model="meta-llama/Llama-2-7b-hf",
    schema_builder=SchemaBuilder(sample_input, sample_output),
    model_path=path, #local path where artifacts will be saved
    mode=Mode.LOCAL_CONTAINER, # The model will be deployed locally. Change to Mode.SAGEMAKER_ENDPOINT to deploy to a SageMaker endpoint. 
    env_vars={
        "HUGGING_FACE_HUB_TOKEN": "<YourHuggingFaceToken>" # Llama 2 is a gated model and requires a Hugging Face Hub token. 
    }
)

By default, ModelBuilder will use [TGI](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#huggingface-text-generation-inference-containers) container as the underlying container for Hugging Face models. In case you would like to use the [LMI containers](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers), you can configure the ModelBuilder as follow:

```python
from sagemaker.serve import ModelServer

model_builder = ModelBuilder(
    model="meta-llama/Llama-2-7b-hf",
    schema_builder=SchemaBuilder(sample_input, sample_output),
    model_path=path, #local path where artifacts will be saved
    mode=Mode.LOCAL_CONTAINER, # The model will be deployed locally. Change to Mode.SAGEMAKER_ENDPOINT to deploy to a SageMaker endpoint. 
    model_server=ModelServer.DJL_SERVING,
    env_vars={
        "HUGGING_FACE_HUB_TOKEN": "<YourHuggingFaceToken>" # Llama 2 is a gated model and requires a Hugging Face Hub token. 
    }
)
```

In [ ]:
model_1 = model_builder_1.build()

In [ ]:
local_predictor = model_1.deploy()

In [ ]:
inferred_sample_prompt = model_builder_1.schema_builder.sample_input
inferred_sample_prompt

In [ ]:
response = local_predictor.predict(inferred_sample_prompt)
print(response)

### SageMaker ModelBuilder: Deploy to a SageMaker Endpoint - Inference Component

Now that we have tested the model prediction locally, we can continue to deploy the model to a SageMaker endpoint using an Inference Component. Lama2-7b can fit on 1 GPU and as such we will create an Inference Component with 1 accelerator. We will also set the `NUM_SHARD` to `1` as there will only be 1 GPU available for the model to use. 

In [ ]:
model_1.env["NUM_SHARD"] = "1"
model_1.env

In [ ]:
from sagemaker.utils import unique_name_from_base
from sagemaker.enums import EndpointType
from sagemaker.compute_resource_requirements.resource_requirements import ResourceRequirements

resource_requirements = ResourceRequirements(
    requests={
        "num_accelerators": 1, #llama2-7b can fit on 1 GPU
        "memory": 1024,
        "copies": 3, # We will deploy on a ml.g5.12xlarge which has 4 GPUs so lets have 3 copies and leave one slot for the next model
    },
    limits={},
)

model_1.model_name = unique_name_from_base("llama2-7b-ic")

predictor_1 = model_1.deploy(
    instance_type="ml.g5.12xlarge", # Here we set the instance type which has 4 GPUs. If we did not set the instance type Model Builder would auto detect the instance we are running on. 
    mode=Mode.SAGEMAKER_ENDPOINT,
    endpoint_type=EndpointType.INFERENCE_COMPONENT_BASED,
    endpoint_name=unique_name_from_base("model-builder-llama2-7b-ic-endpoint"),
    resources=resource_requirements,
    role=execution_role,
    endpoint_logging=False,

)

In [ ]:
print(predictor_1.predict(inferred_sample_prompt)[0]["generated_text"])

### SageMaker ModelBuilder: Deploy a second model to the SageMaker Endpoint - Inference Component

In [ ]:
prompt = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the"
response = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the east coast."

sample_input = {
    "inputs": prompt,
    "parameters": {}
}

sample_output = [
    {
        "generated_text": response
    }
]

model_builder_2 = ModelBuilder(
    model="meta-llama/Llama-2-7b-hf",
    schema_builder=SchemaBuilder(sample_input, sample_output),
    model_path=path,
    env_vars={
        "HUGGING_FACE_HUB_TOKEN": "<YourHuggingFaceToken>"
    }
)

In [ ]:
model_2 = model_builder_2.build()

In [ ]:
model_2.env["NUM_SHARD"] = "1"
model_2.env

In [ ]:
resource_requirements = ResourceRequirements(
    requests={
        "num_accelerators": 1,
        "memory": 1024,
        "copies": 1,
    },
    limits={},
)

model_2.model_name = unique_name_from_base("lama2-7b-ic-2")

predictor_2 = model_2.deploy(
    mode=Mode.SAGEMAKER_ENDPOINT,
    endpoint_type=EndpointType.INFERENCE_COMPONENT_BASED,
    endpoint_name=predictor_1.endpoint,
    resources=resource_requirements,
    role=execution_role,
    endpoint_logging=False,
)

In [ ]:
print(predictor_2.predict(inferred_sample_prompt))

## Clean up

In [ ]:
predictor_1.delete_predictor(wait=True)
predictor_2.delete_predictor(wait=True)

In [ ]:
predictor_1.delete_endpoint()
try:
    predictor_2.delete_endpoint()
except Exception:
    print("endpoint successfully deleted")